In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import glob
import os
import pickle
import matplotlib.pyplot as plt
import copy
# Printing All Results
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from adjToGraphlist import adjToGraphlist
from TransmissionGraph import TransmissionGraph

In [3]:
############################################################################################################################
##########################################       Loading Data        #######################################################
############################################################################################################################

In [4]:
# Name of the different types of the graphs
GraphNames = ["allVillageRelationships","andRelationships",
             "giveadvice","helpdecision",
             "keroricecome","keroricego",
             "borrowmoney","lendmoney",
             "medic","nonrel",
             "rel","templecompany",
             "visitcome","visitgo"]

In [5]:
%%time
## Location to save graphs
path_out = r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\Graphs/'
path_in = r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\IndianVillagesData/'

## Creating a graphlist for each graph name and then save that
for name in GraphNames:
    print(name)
    x = adjToGraphlist(name = name,path=path_in)
    with open(path_out+name+"_Graphs", 'wb') as fp:
        pickle.dump(x, fp) 

allVillageRelationships
andRelationships
giveadvice
helpdecision
keroricecome
keroricego
borrowmoney
lendmoney
medic
nonrel
rel
templecompany
visitcome
visitgo
Wall time: 1min 57s


In [6]:
%%time
## Reading Graphs
path = r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\Graphs/'

with open(path+"allVillageRelationships_Graphs", 'rb') as fp:
    ALL=pickle.load(fp)
    
with open(path+"andRelationships_Graphs", 'rb') as fp:
    AND=pickle.load(fp)
    
with open(path+"giveadvice_Graphs", 'rb') as fp:
    giveadvice=pickle.load(fp)
    
with open(path+"helpdecision_Graphs", 'rb') as fp:
    helpdecision=pickle.load(fp)
    
with open(path+"keroricecome_Graphs", 'rb') as fp:
    keroricecome=pickle.load(fp)
    
with open(path+"keroricego_Graphs", 'rb') as fp:
    keroricego=pickle.load(fp)
    
with open(path+"borrowmoney_Graphs", 'rb') as fp:
    borrowmoney=pickle.load(fp)
    
with open(path+"lendmoney_Graphs", 'rb') as fp:
    lendmoney=pickle.load(fp)
    
with open(path+"medic_Graphs", 'rb') as fp:
    medic=pickle.load(fp)
    
with open(path+"nonrel_Graphs", 'rb') as fp:
    nonrel=pickle.load(fp)
    
with open(path+"rel_Graphs", 'rb') as fp:
    rel=pickle.load(fp)
    
with open(path+"templecompany_Graphs", 'rb') as fp:
    templecompany=pickle.load(fp)
    
with open(path+"visitcome_Graphs", 'rb') as fp:
    visitcome=pickle.load(fp)
    
with open(path+"visitgo_Graphs", 'rb') as fp:
    visitgo=pickle.load(fp)

Wall time: 1.42 s


In [7]:
%%time
## Composing Graphs for different kind of graphs and the set the weight and lebel attribute

Money_Graph = list()
for i in range(75):
    U = nx.compose(lendmoney[i],borrowmoney[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Money"
    Money_Graph.append(U)
    
    
    
Visit_Graph = list()
for i in range(75):
    U = nx.compose(visitcome[i],visitgo[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Visit"
    Visit_Graph.append(U)
    
    
    
Kerosene_Graph = list()
for i in range(75):
    U = nx.compose(keroricecome[i],keroricego[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Kerosene"
    Kerosene_Graph.append(U)
    
    
    
Temple_Graph = list()
for i in range(75):
    U = templecompany[i]
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Temple"
    Temple_Graph.append(U)

Wall time: 2.61 s


In [8]:
%%time
## Creating Multigraph
GraphList = list()
for i in range(75):
    G = nx.MultiGraph()
    G.add_edges_from(Money_Graph[i].edges(data=True))
    G.add_edges_from(Visit_Graph[i].edges(data=True))
    G.add_edges_from(Kerosene_Graph[i].edges(data=True))
    G.add_edges_from(Temple_Graph[i].edges(data=True))
    G.add_nodes_from(Money_Graph[i].nodes())
    
    # For normalizing centrality attribute
    lng = len(G.nodes()) - 1
    
    # Normalizing centrality attribute (from fraction to integer)
    Money_DegCen = nx.degree_centrality(Money_Graph[i])
    Money_DegCen.update((x , int(y*lng))for x, y in Money_DegCen.items())
    # Setting attributes for each node
    nx.set_node_attributes(G, Money_DegCen, 'Money_DegCen')
    
    Visit_DegCen = nx.degree_centrality(Visit_Graph[i])
    Visit_DegCen.update((x , int(y*lng))for x, y in Visit_DegCen.items())
    nx.set_node_attributes(G, Visit_DegCen, 'Visit_DegCen')
    
    Kerosene_DegCen = nx.degree_centrality(Kerosene_Graph[i])
    Kerosene_DegCen.update((x , int(y*lng))for x, y in Kerosene_DegCen.items())
    nx.set_node_attributes(G, Kerosene_DegCen, 'Kerosene_DegCen')
    
    Temple_DegCen = nx.degree_centrality(Temple_Graph[i])
    Temple_DegCen.update((x , int(y*lng))for x, y in Temple_DegCen.items())
    nx.set_node_attributes(G, Temple_DegCen, 'Temple_DegCen')
    
    GraphList.append(G)

# Saving Multigraph
path=r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\ResultMultigraphs/'
with open(path+"GraphList_Multi", 'wb') as fp:
    pickle.dump(GraphList, fp) 

Wall time: 2.83 s


In [83]:
#######################################    Load from here!    #####################################

In [9]:
path=r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\ResultMultigraphs/'
with open(path+"GraphList_Multi", 'rb') as fp:
    GraphList = pickle.load(fp) 

In [12]:
%%time
Dict = {"Money":0.3,"Visit":0,"Kerosene":0.1,"Temple":0.25}
# Copying mulitigraph (Otherwise it's chnaged in the each run)
data = [x.copy() for x in GraphList]

# Creating the graph with given P and filteration policy
temp=TransmissionGraph(Data=data,dropNode='',dropTop=0,Label_dict = Dict)

# Saving the graph to run simulatin on
path=r'C:\Users\Mahdi\OneDrive\Vasserman - Akbarpour\Data\ResultsFilteredGraphs/'
with open(path+"GraphList_Drop_V", 'wb') as fp:
    pickle.dump(temp, fp) 

Wall time: 13.7 s


In [14]:
## Checking the output
i=34
print('number of nodes: before={}, after={}'.format(GraphList[i].number_of_nodes(),temp[i].number_of_nodes()))
print('number of edges: before={}, before_multi={}, after={}'.format(len(set(GraphList[i].edges())),
                                                len(GraphList[i].edges()),temp[i].number_of_edges()))


u=list(temp[i].edges())[6][0]
v=list(temp[i].edges())[6][1]
temp[i].get_edge_data(u,v)
GraphList[i].get_edge_data(u,v)
Dict

number of nodes: before=121, after=121
number of edges: before=446, before_multi=895, after=352


{'weight': 0.3}

{0: {'weight': 1, 'label': 'Money'}, 1: {'weight': 1, 'label': 'Visit'}}

{'Money': 0.3, 'Visit': 0, 'Kerosene': 0.1, 'Temple': 0.25}